In [1]:
from PythonCode.preprocess import *

In [2]:
from PythonCode.models import *

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [ ]:
preprocess_pipeline("Data/C50train/", "bag of words", 0.3,
                        ("x_train_2.csv","x_test_2.csv","y_train_2.csv","y_test_2.csv"))